In [1]:
import os
from pathlib import Path
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
groq_api_key = os.environ.get("GROQ_API_KEY")

In [3]:
# Loading/Reading all the pdf documents

from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader

loader = DirectoryLoader(
    "../data/pdf_files",
    glob="**/*.pdf",
    loader_cls=PyPDFLoader
)

docs = loader.load()
len(docs)


56

In [4]:
# Splitting the Texts into Chunks

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    separators=["\n\n", "\n", " ", ""]
)

split_texts = text_splitter.split_documents(docs)
print(f"Created {len(split_texts)} chunks")

Created 98 chunks


In [5]:
# Embedding the split texts into vectors

from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

# Creating the Vector Store (ChromaDB)

from langchain_community.vectorstores import Chroma

vectorstores = Chroma.from_documents(
    documents=split_texts,
    embedding=embeddings,
    persist_directory="../data/chroma_db"
)

print(f"Vector store created with {len(split_texts)} embeddings")

C:\Users\Admin\AppData\Local\Temp\ipykernel_1584\2335338193.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(



Vector store created with 98 embeddings


In [6]:
retriever = vectorstores.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}  # Return top 3 most relevant chunks
)

In [7]:
# Adding Memory

from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
    output_key="answer"
)

C:\Users\Admin\AppData\Local\Temp\ipykernel_1584\2105759968.py:5: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [8]:
# Prompting

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an expert AI assistant specialized in construction tender analysis.

Your role is to help analyze tender documents and company capabilities to determine bid feasibility.

IMPORTANT RULES:
- Answer naturally and conversationally (never say "according to the document")
- Use the conversation history to understand context and follow-up questions
- Be specific with numbers, dates, and requirements
- If you don't have enough information, say so clearly
- Focus on actionable insights for decision-making
- Compare tender requirements against company capabilities when relevant

Remember: You're helping a construction company decide whether to bid on projects."""),
    
    MessagesPlaceholder(variable_name="chat_history"),
    
    ("human", """Context information from documents:
{context}

Question: {question}""")
])

from langchain_groq import ChatGroq

groq_llm = ChatGroq(
    groq_api_key=groq_api_key,
    model_name="llama-3.3-70b-versatile",
    temperature=0.1,  # Low temperature for factual responses
    max_tokens=1024
)

print("Connected to Groq LLM (llama-3.3-70b-versatile)")

Connected to Groq LLM (llama-3.3-70b-versatile)


In [9]:
# Build RAG Chain

from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

def format_docs(docs):
    """Format retrieved documents into a single string"""
    return "\n\n".join([doc.page_content for doc in docs])

def get_chat_history(memory):
    """Extract chat history from memory"""
    return memory.load_memory_variables({}).get("chat_history", [])

rag_chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough(),
        "chat_history": lambda x: get_chat_history(memory)
    }
    | prompt
    | groq_llm
    | StrOutputParser()
)

In [10]:
def chat_with_memory(question):
    try:
        # Get answer from RAG chain
        answer = rag_chain.invoke(question)
        
        # Save to memory
        memory.save_context(
            {"input": question},
            {"answer": answer}
        )
        
        return answer
    
    except Exception as e:
        return f"Error generating response: {e}"

In [11]:
response1 = chat_with_memory("What is the Lekki Bridge project about?")
print(response1)

The Lekki Bridge project is a 2.4-kilometer reinforced concrete bridge construction project along the Lekki-Epe Expressway coastal corridor. The goal of the project is to reduce traffic congestion and improve coastal connectivity as part of the Lagos State Infrastructure Development Initiative. The project involves a range of responsibilities, including detailed engineering design, site preparation, construction of the bridge foundation, erection of the superstructure, installation of drainage and safety systems, and more. The bridge will span from Admiralty Way Junction to Chevron Drive, crossing the tidal lagoon section. 

To determine if this project is a good fit for your company, I'd like to know more about your company's capabilities and experience with similar projects. What's your company's background in bridge construction, and have you worked on projects of this scale before?


In [12]:
response2 = chat_with_memory("What are the key eligibility requirements for bidders?")
print(response2)

To be eligible to bid on the Lekki Bridge project, companies must meet two main sets of requirements: Company Registration Requirements and Technical Requirements.

Under Company Registration Requirements, bidders must possess:
1. A valid Corporate Affairs Commission (CAC) registration certificate
2. A Tax Clearance Certificate for the current year (2025)
3. Evidence of registration with the Federal Inland Revenue Service (FIRS)
4. A VAT registration certificate
5. A Pension Commission (PenCom) compliance certificate

In terms of Technical Requirements, bidders must have:
1. A minimum of 10 years of operational experience in civil engineering construction
2. Successful completion of at least three bridge construction projects within the last 7 years

These requirements are mandatory, and bidders must meet all of them to be considered for the contract award. The contract will be awarded to the bidder with the highest combined technical and financial score, provided they meet all the eli

In [17]:
response3 = chat_with_memory("Does Elalan Construction meet these requirements? Dont forget about the Third Mainland Bridge Project")
print(response3)

With the additional context information, I can now confirm that Elalan Construction Limited meets the requirements for the Lekki Bridge project.

Firstly, regarding company registration, although the document does not explicitly mention their registration status, it can be inferred that Elalan Construction Limited is a registered company, as they have been operating since 2001 and have worked on significant projects, including the Third Mainland Bridge rehabilitation project.

On the technical side, Elalan Construction Limited has demonstrated exceptional technical competence and project management skills, as evidenced by the formal feedback from the Federal Ministry of Works and Housing (FMWH) in October 2018. The feedback commends their proactive approach to traffic management and safety, and states that they would not hesitate to engage Elalan for future projects of similar scale and complexity.

The Third Mainland Bridge rehabilitation project, which was completed 2 months ahead of